# XADC

This example provides access to slow analog inputs on the E2 connector. Measurements are performed by the integrated Zynq XADC.

In [ ]:
from redpitaya import *

rp = redpitaya()

# number of channels
chn = 4

for ch in range(chn):
    val = rp.AIpinGetValue(ch)
    print("Measured voltage on AI[%d] = {%f} V" % (ch, val))

In [ ]:
import time

import numpy as np
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import HoverTool, Range1d
from bokeh.plotting import figure
output_notebook()

## Measurement logging

This example is using a Pyhon list for the data sequence, which can become a performance issue for large data sets.

A better implementation would use a fixed `numpy` array for display data, and the rest of the data would be stored into a file.

In [ ]:
x = [0]
y = [[rp.AIpinGetValue(ch)] for ch in range(chn)]

colors = ('red', 'blue', 'green', 'orange')
hover = HoverTool(mode = 'vline', tooltips=[("T", "@x"), ("V", "@y")])
tools = "pan,wheel_zoom,box_zoom,reset,crosshair"
p = figure(plot_height=400, plot_width=900, title="XADC log", toolbar_location="above", tools=(tools, hover))
p.xaxis.axis_label='time [s]'
p.y_range=Range1d(0, 5)
p.yaxis.axis_label='voltage [V]'
r = [p.line(x, y[ch], line_width=1, line_alpha=0.7, color=colors[ch]) for ch in range(chn)]

# get and explicit handle to update the next show cell with
target = show(p, notebook_handle=True)

A sample is taken every `T` seconds.
This implementation suffers from time drift,
since the actual period is larger than `T`
by the time it takes to execute the rest of the loop.

In [ ]:
T = 1
#while True:
for i in range(200):
    x.append(x[-1]+T)
    for ch in range(chn):
        r[ch].data_source.data['y'].append(rp.AIpinGetValue(ch))
    # push updates to the plot continuously using the handle (intererrupt the notebook kernel to stop)
    push_notebook(handle=target)
    time.sleep(T)